### Querying data from NHS Scotland Open Data portal using Python

This notebook walks through an example task that involves querying multiple datasets (or resources) and linking them with reference datasets.

If you're planning to use it on NSS internal network, you must launch Jupyter lab from the command line (not Power Shell) and set NSS proxies to make sure outgoing connections go through the firewall. If you're unsure how to do this or have any other questions, don't hesitate to email NSS Python Usergroup inbox <nss.pythonusergroupinbox@nhs.net>.

In [1]:
import requests
import pandas as pd #check your Pandas version; this notebook was written using 0.25.1

#### This tutorial uses SQL to query resources and do basic analysis prior to downloading the data

In [2]:
#Open Data base URL
url = r"https://www.opendata.nhs.scot/api/3/action/datastore_search_sql?sql="

### HB Lookup

In [3]:
resource_id = "652ff726-e676-4a20-abda-435b98dd7bdc"

In [4]:
#column and table names (saved in the resource_id variable) must be put in extra quotes
sql = f'SELECT "HB2014", "HB2014Name" FROM "{resource_id}"'

In [5]:
final_url = url + sql

The "success" response code is `200`. You might also get a `timeout` error, meaning there is a problem with your connection, most likely to do with proxies, or a `409` code, meaning you sql code isn't valid.

In [6]:
response = requests.get(final_url)

In [7]:
hb_lookup = pd.DataFrame(response.json()['result']['records'])

### GP Practice Lookup

In [8]:
resource_id = "c01dc5f3-86ea-4a3d-8e0c-1d29f04a85d7"

In [9]:
sql = f'SELECT "PracticeCode", "GPPracticeName", "NHSBoard" FROM "{resource_id}"'

In [10]:
final_url = url + sql
response = requests.get(final_url)

In [11]:
gp_lookup = pd.DataFrame(response.json()['result']['records'])

In [12]:
#pick 10 GP practices per board and format for SQL IN filter
gp_filter = ','.join(gp_lookup.groupby('NHSBoard')['NHSBoard', 'PracticeCode'].head(10).PracticeCode.values)

### Put together monthly Prescribing data

In [13]:
resources = [
    "5c667230-4201-4a09-949d-3e6cc3a4ec19", #August
    "6e3856e9-88cb-495a-8c8a-54b0460df950", #July
    "6ea2f299-76bc-49cd-ab43-9228b601da5f", #June
    "7479536b-0b95-43d6-9152-31bbd522e6b4", #May
    "02197246-5d98-4ba9-b25d-218ac9cd91e6"  #April
]

In [14]:
dfs = []

for resource in resources:
    sql = f"""
SELECT "HBT2014", "GPPractice", "BNFItemCode", "BNFItemDescription", "NumberOfPaidItems", "PaidDateMonth", "PaidQuantity"
FROM "{resource}"
WHERE "GPPractice" IN ({gp_filter})
AND "BNFItemCode" NOT LIKE ' '
AND ("BNFItemCode" IN (
    SELECT "BNFItemCode"
    FROM "{resource}"
    WHERE "GPPractice" IN ({gp_filter})
    GROUP BY "BNFItemCode"
    ORDER BY SUM("GrossIngredientCost")  DESC, "BNFItemCode"
    LIMIT 5)
    OR
    "BNFItemCode" IN (
    SELECT "BNFItemCode"
    FROM "{resource}"
    WHERE "GPPractice" IN ({gp_filter})
    GROUP BY "BNFItemCode"
    ORDER BY SUM("GrossIngredientCost")  ASC, "BNFItemCode"
    LIMIT 5)   
    )
"""
    final_url = url + sql
    response = requests.get(final_url)
    temp_df = pd.DataFrame(response.json()['result']['records'])
    temp_df.PaidDateMonth = pd.to_datetime(temp_df.PaidDateMonth, format="%Y%m")
    temp_df.NumberOfPaidItems = temp_df.NumberOfPaidItems.astype(float)
    temp_df.PaidQuantity = temp_df.PaidQuantity.astype(float)
    
    dfs.append(temp_df)

In [15]:
final_coded = pd.concat(dfs)

### Link the HB and GP Practice Code with Names

In [16]:
hb_linked = pd.merge(final_coded, hb_lookup, how='left', left_on='HBT2014', right_on='HB2014')
final = pd.merge(hb_linked, gp_lookup, how='left', left_on='GPPractice', right_on='PracticeCode')
final.drop(columns=['HB2014', 'HBT2014', 'GPPractice', 'PracticeCode', 'NHSBoard'], inplace=True)

In [17]:
#re-order columns
final = final[['PaidDateMonth', 'HB2014Name','GPPracticeName', 'BNFItemCode','BNFItemDescription', 'NumberOfPaidItems', 'PaidQuantity']]

In [18]:
#preview the data
final.head()

,PaidDateMonth,HB2014Name,GPPracticeName,BNFItemCode,BNFItemDescription,NumberOfPaidItems,PaidQuantity
0,2019-08-01,NHS Tayside,MUIRHEAD MEDICAL CENTRE,21480000100,FREESTYLE LIBRE SENSOR KIT,14.0,36.0
1,2019-08-01,NHS Tayside,THE BLUE PRACTICE,0208020Y0BBACAC,XARELTO_TAB 20MG,12.0,532.0
2,2019-08-01,NHS Tayside,THE BLUE PRACTICE,0208020Z0BBABAB,ELIQUIS_TAB 5MG,12.0,1008.0
3,2019-08-01,NHS Tayside,THE BLUE PRACTICE,0302000C0BQAABX,FOSTAIR_INH 100MCG/6MCG (120D) CFF,63.0,81.0
4,2019-08-01,NHS Tayside,THE BLUE PRACTICE,21480000100,FREESTYLE LIBRE SENSOR KIT,10.0,34.0


In [19]:
#export to current directory
final.to_csv('prescribing.csv', index=False)